In [1]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
openai.api_type = 'azure'
openai.api_base = 'https://nous.openai.azure.com/'
openai.api_version = '2023-07-01-preview'
openai.api_key = ''
engine = 'nous-16k'

In [3]:
def predict(messages):
    try:
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=1.0,
            max_tokens=1024,
            top_p=1.0
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [4]:
system = 'Above is a conversation between a user and an intelligent assistant. You suppose to simulate conversation between the user and the assistant. Bear in mind your major request is to ask the assistant to generate some material. So you can ask the assistant either to make it more detailed, add more related information, or any other request to improve the generated material. Be creative and diverse in your request. Make the response short and the language casual.'
system_malay = 'Above is a conversation between a user and an intelligent assistant. You suppose to simulate conversation between the user and the assistant in malay language. Bear in mind your major request is to ask the assistant to generate some material. So you can ask the assistant either to make it more detailed, add more related information, or any other request to improve the generated material. Be creative and diverse in your request. Make the response short and the language casual.'
next_step = 'Above is a conversation between a user and an intelligent assistant. Now suppose you are the user, generate response according to the generated material to continue the conversation.'

In [5]:
!mkdir ultrachat

mkdir: cannot create directory ‘ultrachat’: File exists


In [6]:
def ultrachat(row, n = 1):
    results = [
        {'role': 'context', 'content': row['paragraph']},
        {'role': 'user', 'content': row['question']},
    ]
    initial = f"""
    {row['paragraph']}

    {row['question']}
    """.strip()
    messages = [
        {'role': 'system', 'content': system_malay},
        {'role': 'user', 'content': initial},
    ]
    r = predict(messages)
    results.append({
        'role': 'assistant', 'content': r,
    })
    messages.append({
        'role': 'assistant', 'content': r,
    })
    
    for _ in range(n):
    
        messages_temp = messages + [
            {'role': 'user', 'content': 'Now suppose you are the user, say something to continue the conversation based on given context. Make the response short and the language casual'}
        ]
        r2 = predict(messages_temp)
        if r2 is None:
            break

        results.append({
            'role': 'user', 'content': r2,
        })
        messages.append({
            'role': 'user', 'content': r2,
        })

        r = predict(messages)
        if r is None:
            break
        results.append({
            'role': 'assistant', 'content': r,
        })
        messages.append({
            'role': 'assistant', 'content': r,
        })
    
    return results

In [7]:
from glob import glob

files = sorted(glob('question/*.json'))
len(files)

10128

In [8]:
def generate(f):
    index = int(os.path.split(f)[1].replace('.json', ''))
    filename = f'ultrachat/{index}.json'
    if os.path.exists(filename):
        return
        
    with open(f) as fopen:
        data = json.load(fopen)
    
    if len(data['paragraph'].split()) < 100:
        return
    
    r = ultrachat(data, n = random.randint(2, 5))
    with open(filename, 'w') as fopen:
        json.dump(r, fopen)

In [10]:
generate(files[-3])

In [12]:
with open('ultrachat/9997.json') as fopen:
    data = json.load(fopen)
    
data

[{'role': 'context',
  'content': '30% Nil 8418.29 - - Other: 300 - - - Not exceeding 350 l capacity u 30% Nil 900 - - - Other u 30% Nil 8418.30 - Freezers of the chest type, not exceeding 800 l capacity: 200 - - Not exceeding 200 l capacity u 30% Nil 900 - - Other u 30% Nil 8418.40 - Freezers of the upright type, not exceeding 900 l capacity: 200 - - Not exceeding 200 l capacity u 30% Nil 900 - - Other u 30% Nil 8418.50 000 - Other furniture (chests, cabinets, display counter, show-cases and the like) for storage and display, incorporating refrigerating or freezing equipment u 30% Nil - Other refrigerating or freezing equipment; heat pumps: 8418.61 000 - - Heats pumps other than air conditioning machines of heading 84.15 u 30% Nil 8418.69 000 - - Other kg 30% Nil P.U. (A) 275 542 RATE OF DUTY Heading / subheading Description Unit of Quantity Import Export (1) (2) (3) (4) (5) - Parts: 8418.91 000 - - Furniture designed to receive refrigerating or freezing equipment kg 30% Nil 8418.99 -

In [ ]:
from tqdm import tqdm

max_worker = 2
for i in tqdm(range(0, len(files), max_worker)):
    b = files[i: i + max_worker]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, f): f for f in b}

        for future in as_completed(futures):
            future.result()

 11%|████▏                                | 569/5064 [17:13<42:38:30, 34.15s/it]

'content'


 20%|██████▊                            | 994/5064 [3:38:21<32:48:10, 29.01s/it]

'content'


 22%|██████▉                         | 1093/5064 [4:33:11<219:24:55, 198.92s/it]

...

 32%|██████████▊                       | 1618/5064 [9:00:23<24:51:56, 25.98s/it]

'content'


 35%|███████████▍                     | 1750/5064 [10:00:17<21:48:51, 23.70s/it]

Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


 36%|███████████▊                     | 1804/5064 [10:26:17<22:51:39, 25.25s/it]

In [ ]:
with open('ultrachat/0.json') as fopen:
    data = json.load(fopen)

In [ ]:
data